<a href="https://www.kaggle.com/code/amirulmahmud/heart-failure-prediction-with-logistic-regression?scriptVersionId=124935993" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# The Objective

The objective of this work is to create a Classification Model that can predict whether or not a person has presence of heart failure based on physical features of that person.

# Load The Data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
df = pd.read_csv('/kaggle/input/heart-failure-prediction/heart.csv')

In [ ]:
df.head()

In [ ]:
df.info()

# Data Cleaning

Before data is used, let's check if there is any missing values in the data.

In [ ]:
df.isna().sum()

Fortunately, the data is complete and does not have any missing values.

# Exploratory Data Analysis

**Create a statistical summary**

In [ ]:
df.describe().transpose()

**Create a pairplot**

In [ ]:
sns.pairplot(df,hue='HeartDisease')

**Create a heatmap that displays the correlation between features**

In [ ]:
plt.figure(figsize=(6,4),dpi=150)
sns.heatmap(df.corr(),cmap='viridis',annot=True)

**Find the top 5 correlated features with target.**

In [ ]:
np.abs(df.corr()['HeartDisease']).sort_values().tail(6)

# Split The Data

The approach here will use Cross Validation on 90% of the dataset, and then judge the results on a final test set of 10% to evaluate the model.

In [ ]:
X = df.drop('HeartDisease',axis=1)
y = df['HeartDisease']

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [ ]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

Check the balance of the label data in training set.

In [ ]:
y_train.value_counts()

In [ ]:
sns.countplot(x=y_train)

In [ ]:
372/(454+372)

As we see here, the label class of training data is slightly balance (45% : 55%). So, the data is ready to use in the next steps.

# Features Engineering

ColumnTransformer is particularly handy for the case of datasets that contain heterogeneous data types, since we may want to scale the numeric features and one-hot encode the categorical ones.

In [ ]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

In [ ]:
df.info()

In [ ]:
df['FastingBS'].value_counts()

In [ ]:
col_numeric = ['Age','RestingBP','Cholesterol','MaxHR','Oldpeak']
col_categoric = ['Sex','ChestPainType','FastingBS','RestingECG','ExerciseAngina','ST_Slope']

In [ ]:
scaler = StandardScaler()
encoder = OneHotEncoder(drop='first')

In [ ]:
preprocessor = ColumnTransformer([
    ('num',scaler,col_numeric),
    ('cat',encoder,col_categoric)
])

# Logistic Regression Model

**Create a base model of Logistic Regression**

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
base_model = LogisticRegression(max_iter=1000)

**Create a Pipeline**

In [ ]:
from sklearn.pipeline import Pipeline

In [ ]:
pipe = Pipeline([('preprocessor',preprocessor),('base_model',base_model)])

Perform a grid-search with the pipeline to test various parameters and report back the best performing parameters.

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
parameters = ({'base_model__C': [0.001,0.01,0.1,1,10],
               'base_model__penalty': ['l1','l2','elasticnet'],
               'base_model__solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
              })

In [ ]:
grid = GridSearchCV(estimator=pipe,param_grid=parameters,scoring='accuracy',cv=5)

In [ ]:
grid.fit(X_train,y_train)

Find the best estimator and best parameters

In [ ]:
grid.best_estimator_

In [ ]:
grid.best_estimator_.get_params()

# Cross Validation Results

In [ ]:
cv_results = pd.DataFrame(grid.cv_results_)

In [ ]:
cv_results.info()

**Result: C Values**

In [ ]:
cv_c = cv_results.groupby('param_base_model__C').agg('mean')['mean_test_score']
cv_c

**Result : Penalty**

In [ ]:
cv_penalty = cv_results.groupby('param_base_model__penalty').agg('mean')['mean_test_score']
cv_penalty

**Result: Solver**

In [ ]:
cv_solver = cv_results.groupby('param_base_model__solver').agg('mean')['mean_test_score']
cv_solver

# Final Model Evaluation

In [ ]:
y_pred = grid.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix,plot_confusion_matrix

In [ ]:
confusion_matrix(y_test,y_pred)

In [ ]:
print(classification_report(y_test,y_pred))

In [ ]:
plot_confusion_matrix(grid,X_test,y_test)

In [ ]:
from sklearn.metrics import accuracy_score,f1_score

In [ ]:
accuracy_score(y_test,y_pred)

In [ ]:
f1_score(y_test,y_pred)

# CONCLUSION

1. The best parameters of Logistic Regression in this model are combination of C = 1, penalty = 'l2', and solver = 'newton-cg'.
1. The model performs quite well in predicting the unseen data (X_test), with accuracy of 84.78% and f1 score of 86.79%.